In [ ]:
!pip install fastapi uvicorn pyngrok langchain-groq langchain nest_asyncio jinja2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 11.6 MB/s eta 0:00:00


In [ ]:

import os
import nest_asyncio
from pyngrok import ngrok
from fastapi import FastAPI, Request, Form
from fastapi.responses import HTMLResponse
from fastapi.templating import Jinja2Templates
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_groq import ChatGroq



In [ ]:
GROQ_API_KEY = "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"  # replace with your key
NGROK_AUTH_TOKEN = "316pNuAwhjYD8giBKQblyXolHAf_6r1FYADkHEz72xGogbbik"  # replace with your ngrok token

os.environ["GROQ_API_KEY"] = GROQ_API_KEY
ngrok.set_auth_token(NGROK_AUTH_TOKEN)



In [ ]:
code = '''
import os
from fastapi import FastAPI, Request, Form
from fastapi.responses import HTMLResponse
from fastapi.templating import Jinja2Templates
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_groq import ChatGroq

app = FastAPI()
templates = Jinja2Templates(directory="templates")

llm = ChatGroq(
    model="llama3-8b-8192",
    api_key=os.environ["GROQ_API_KEY"],
    temperature=0.7
)

memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(llm=llm, memory=memory)

@app.get("/", response_class=HTMLResponse)
async def myhome(request: Request):
    return templates.TemplateResponse("chat.html", {"request": request, "chat_history": []})

@app.post("/", response_class=HTMLResponse)
async def chat(request: Request, message: str = Form(...)):
    conversation.predict(input=message)
    history = []
    for msg in memory.chat_memory.messages:
        sender = 'You' if msg.type == 'human' else 'Bot'
        history.append((sender, msg.content))
    return templates.TemplateResponse("chat.html", {"request": request, "chat_history": history})
'''

with open("main.py", "w") as f:
    f.write(code)

html_code = """
<html>
<head>
    <title>LangChain Chatbot (LLaMA 3 via Groq)</title>
    <style>
        body { font-family: Arial; width: 50%; margin: auto; background-color: #f8f9fa; }
        .user { color: blue; }
        .bot { color: green; }
        .chat-box { border: 1px solid #ccc; padding: 10px; background: white; margin-bottom: 10px; min-height: 200px; }
    </style>
</head>
<body>
    <h1>LangChain Chatbot (LLaMA 3 via Groq)</h1>
    <div class="chat-box">
        {% for sender, text in chat_history %}
            <p class="{{ 'user' if sender == 'You' else 'bot' }}"><b>{{ sender }}:</b> {{ text }}</p>
        {% endfor %}
    </div>
    <form method="post">
        <input type="text" name="message" placeholder="Type your message" style="width:80%">
        <button type="submit">Send</button>
    </form>
</body>
</html>
"""
os.makedirs("templates", exist_ok=True)
with open("templates/chat.html", "w") as f:
    f.write(html_code)

nest_asyncio.apply()
public_url = ngrok.connect(8000)
print("Public URL:", public_url)

!uvicorn main:app --host 0.0.0.0 --port 8000 --reload


Public URL: NgrokTunnel: "https://2ef72967424c.ngrok-free.app" -> "http://localhost:8000"
INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [3173] using StatReload
/content/main.py:19: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)
/content/main.py:20: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversation = ConversationChain(llm=llm, memory=memory)
INFO:     Started server process [3175]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     197.58.76.127:0 - "GET / HTTP/1.1" 200 OK
INFO:     197.58.76.127:0 - "GET /favicon.ico HT

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [3175]
INFO:     Stopping reloader process [3173]
